In [90]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

"""
Closing Price
Represents Sj(t)
"""


def closing_price(stockDF, T):
    return stockDF.loc[str(T), 'Close']


def daily_rate_of_return(stockDF, T):
    today = closing_price(stockDF, T)
    yesterday = closing_price(stockDF, T - 1)
    return (today - yesterday) / yesterday


def vj(stockDF, T):
    rates_of_return = []
    for i in range(0, 15):
        rates_of_return.append(daily_rate_of_return(stockDF, T - i))
    # Invert the elements in rates_of_return
    rates_of_return = rates_of_return[:: -1]
    return rates_of_return


# Try this both ways before submitting
def true_class(targ):
    if targ >= 0.006:
        return "UP"
    elif targ <= -0.006:
        return "DOWN"
    else:
        return "STABLE"


def targ(stockDF, T):
    return daily_rate_of_return(stockDF, T+1)


def make_zt(X):
    return X.map(true_class)

def make_xt(stocks, T):
    """
    Xt Represents an array containing all our line vector stock data "Vj(t)"
    """
    xt_list = []
    for stock, ticker in stocks:
        xt_list.append((ticker, vj(stock, T)))
    return xt_list


In [40]:
tickers = ["AAPL", "MSFT", "TSLA", "META", "GOOGL", "AMZN", "NVDA", "AMD", "DIS", "NFLX",
           "JPM", "KO", "BAC", "C", "WFC", "GS", "AXP", "MCD", "DJI", "SPY"]

stocks = []

for ticker in tickers:
    stocks.append((yf.Ticker(ticker).history(start='2016-01-01', end='2022-12-31'), ticker))

for stock in stocks:
    new_index = [str(i + 1) for i in range(len(stock[0]))]
    stock[0].index = new_index


In [41]:
Y = []
for stock, ticker in stocks:
    for day in range(2, len(stock) - 1):
        Y.append((ticker, day, daily_rate_of_return(stock, day)))

In [42]:
import matplotlib.cm as cm

colors = cm.get_cmap('rainbow', 3)  # using 'rainbow' colormap here, you can choose any
unique_tickers = list(set([x[0] for x in Y]))  # gets unique tickers


# Chunking function
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


# Dividing tickers into chunks of 5
chunks_of_tickers = list(chunks(unique_tickers, 3))
"""
# Now plotting
for chunk in chunks_of_tickers:
    for i, ticker in enumerate(chunk):
        ticker_data = [x for x in Y if x[0] == ticker]  # filter out data for this ticker only
        days = [x[1] for x in ticker_data]  # x-axis data
        rates_of_return = [x[2] for x in ticker_data]  # y-axis data
        plt.plot(days, rates_of_return, color=colors(i), label=ticker)
    plt.xlabel('Day')
    plt.ylabel('Rate of Return')
    plt.title('Rate of Return Over Days for Each Ticker')
    plt.legend()  # add a legend
    plt.show()
"""



/tmp/ipykernel_450546/358189265.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = cm.get_cmap('rainbow', 3)  # using 'rainbow' colormap here, you can choose any


"\n# Now plotting\nfor chunk in chunks_of_tickers:\n    for i, ticker in enumerate(chunk):\n        ticker_data = [x for x in Y if x[0] == ticker]  # filter out data for this ticker only\n        days = [x[1] for x in ticker_data]  # x-axis data\n        rates_of_return = [x[2] for x in ticker_data]  # y-axis data\n        plt.plot(days, rates_of_return, color=colors(i), label=ticker)\n    plt.xlabel('Day')\n    plt.ylabel('Rate of Return')\n    plt.title('Rate of Return Over Days for Each Ticker')\n    plt.legend()  # add a legend\n    plt.show()\n"

In [301]:
Xt = make_xt(stocks, 17)

# extract vj values and convert to dataframe
X = []
for ticker, rates in Xt:
    for rate in rates:
        X.append(rate)

X_data = pd.DataFrame([X], columns=[f'feature_{i}' for i in range(1, len(X) + 1)])

# Now each entry in X will be a separate column in X_data
Z_data = make_zt(X_data)


In [97]:
from sklearn.ensemble import RandomForestClassifier
import time
# Create our random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True)

start_time = time.time()
clf.fit(X_data, Z_data)
end_time = time.time()

oob_score = clf.oob_score_

Z_pred = clf.predict(X_data)
elapsed_time = end_time - start_time

print(f"Training took {elapsed_time} seconds.")

print(f"OOB : {oob_score}")

Training took 0.9359517097473145 seconds.
OOB : 1.0


/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [302]:
stable_columns = (Z_data == 'STABLE').all()
CL0 = Z_data.loc[:, stable_columns]

up_columns = (Z_data == 'UP').all()
CL1 = Z_data.loc[:, up_columns]

down_columns = (Z_data == 'DOWN').all()
CL2 = Z_data.loc[:, down_columns]

s0 = CL0.shape[1]
print(s0)
s1 = CL1.shape[1]
print(s1)
s2 = CL2.shape[1]
print(s2)

N = Z_data.shape[1]


ratio0 = s0/N
ratio1 = s1/N
ratio2 = s2/N

print(f"Stable Ratio: {ratio0}\nUp Ratio: {ratio1}\nDown Ratio: {ratio2}")

68
88
144
Stable Ratio: 0.22666666666666666
Up Ratio: 0.29333333333333333
Down Ratio: 0.48


In [333]:
import numpy as np
## Increase the number of Stable and Up days
import numpy as np

percent_increase = 0.65
percent_increase_stable = 1.00

# Applying condition on all columns and count True results, then scale
n_up = int((X_data >= 0.006).sum().sum() * percent_increase)
n_stable = int(((X_data < 0.006) & (X_data > -0.006)).sum().sum() * percent_increase_stable)
print(f"Number of 'UP' entries to add {n_up}")
print(f"Number of 'STABLE' entries to add {n_stable}")


Number of 'UP' entries to add 57
Number of 'STABLE' entries to add 68


In [334]:
# Identify columns where maximum value is >= 0.006
eligible_columns = X_data.apply(lambda col: col.max() > 0.0059)
# Filter those columns
filtered_X_data = X_data.loc[:, eligible_columns]

# Sample n_up columns with replacement 
sampled_columns = filtered_X_data.sample(n=n_up, replace=True, axis=1)

# Select the first row and transpose it to be 1 x n_up
X_up = sampled_columns.iloc[0].T

# Make it dataframe and transpose to align to original form
X_up = pd.DataFrame(X_up).T
# Rename the columns
X_up.columns = ["cloned_up_" + str(i + 1) for i in range(X_up.shape[1])]
# Update the row labels
X_up = X_up.reset_index(drop=True)
print(X_up.shape)


(1, 57)


In [335]:

# Identify columns where values are between -0.006 and 0.006
eligible_columns = X_data.apply(lambda col: -0.006 < col.max() < 0.006)

# Filter those columns
filtered_X_data = X_data.loc[:, eligible_columns]

# Sample n_stable columns with replacement
sampled_columns = filtered_X_data.sample(n=n_stable, replace=True, axis=1)

# Select the first row and transpose it to be 1 x n_stable
X_stable = sampled_columns.iloc[0].T

# Make it dataframe and transpose to align to original form
X_stable = pd.DataFrame(X_stable).T

X_stable.columns = ["cloned_stable_" + str(i + 1) for i in range(X_stable.shape[1])]
print(X_stable.shape)

(1, 68)


In [336]:
X_extended = pd.concat([X_data, X_up, X_stable], axis=1)
print(X_extended.shape)


(1, 425)


In [337]:
M = X_extended.shape[1]
print(M)
Z_extended = make_zt(X_extended)

# Slide 8
stable_columns = (Z_extended == 'STABLE').all()
CL0 = Z_extended.loc[:, stable_columns]

up_columns = (Z_extended == 'UP').all()
CL1 = Z_extended.loc[:, up_columns]

down_columns = (Z_extended == 'DOWN').all()
CL2 = Z_extended.loc[:, down_columns]

s0 = CL0.shape[1]
print(s0)
s1 = CL1.shape[1]
print(s1)
s2 = CL2.shape[1]
print(s2)


ratio0 = s0/M
ratio1 = s1/M
ratio2 = s2/M

print(f"Altered Stable Ratio: {ratio0}\nAltered Up Ratio: {ratio1}\nAltered Down Ratio: {ratio2}")


425
136
145
144
Altered Stable Ratio: 0.32
Altered Up Ratio: 0.3411764705882353
Altered Down Ratio: 0.3388235294117647


In [340]:
X_train = X_extended.drop(X_extended.columns[1], axis=1)
Z_train = Z_extended.drop(Z_extended.columns[-1], axis=1)

In [341]:
from sklearn.ensemble import RandomForestClassifier
import time
# Create our random forest classifier
random_training_size = int((2*M)/3)
clf = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True)

start_time = time.time()
clf.fit(X_train, Z_train)
end_time = time.time()

oob_score = clf.oob_score_
elapsed_time = end_time - start_time

print(f"Training took {elapsed_time} seconds.")

print(f"OOB : {oob_score}")


Training took 0.7073330879211426 seconds.
OOB : 1.0


/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [363]:
import time

def TrainRandomForest(num_tree, num_select_features):
    from sklearn.ensemble import RandomForestClassifier
    # Create our random forest classifier
    clf = RandomForestClassifier(n_estimators=num_tree, random_state=42, oob_score=True,
                                 bootstrap=True, max_samples=0.87,
                                 max_features=num_select_features, n_jobs=-1)
    start_time = time.time()
    # Drop the first X and last Z so that we have a list of rates of return for 299 days correlated with
    # the 299 up, down, stable labels corresponding to the next day's return
    clf.fit(X_extended, Z_extended)
    end_time = time.time()
    oob_score = clf.oob_score_
    elapsed_time = end_time - start_time
    print(clf.feature_importances_)
    return clf, oob_score, elapsed_time


In [364]:
start_time = time.time()
clf, oob_score, elapsed_time = TrainRandomForest(100, 18)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time was {elapsed_time} seconds")
predicted_z = clf.predict(X_extended)
predicted_z = pd.DataFrame(predicted_z)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [365]:
from sklearn.metrics import confusion_matrix
cmatrix = confusion_matrix(Z_data, predicted_z, labels=['STABLE', 'UP', 'DOWN'])

print(cmatrix)


ValueError: multiclass-multioutput is not supported

In [366]:
trees = [100,200,300,400,500]
selected_features = [18,36,72,140,200]
OOB_accuracies = []
computation_times = []
data = []
for tree in trees:
    for SF in selected_features:
        clf, oob_score, elapsed_time = TrainRandomForest(tree, SF)
        data.append((tree, SF, oob_score, elapsed_time))

# print(data)
df = pd.DataFrame(data, columns=['Tree', 'SF', 'oob_score', 'computation_time'])
df_pivot_oob = df.pivot(index='Tree', columns='SF', values='oob_score')
df_pivot_time = df.pivot(index='Tree', columns='SF', values='computation_time')

print(f"OOB\n {df_pivot_oob}")
print(f"Computation times (s)\n {df_pivot_time}")
# Best TR* = 100; Best SF* = 36

/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


...

/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/home/isaac/Apps/Mara_HW3/.venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [84]:
RF_prime = RandomForestClassifier(n_estimators=500, min_samples_split=18, min_impurity_decrease=0, max_leaf_nodes=12,
                                  max_features=16, criterion='gini', n_jobs=-1, oob_score=True)

Xt_train = X_data[:-1]
Zt_train = Z_data[1:]
RF_prime.fit(Xt_train, Zt_train.values.ravel())

oob_prime = RF_prime.oob_score_
importances = list(RF_prime.feature_importances_)
features = RF_prime.n_features_in_

# Print the feature ranking
print(oob_prime)

print("Feature ranking:")
print(features)
print(importances)


0.4882943143812709
Feature ranking:
1
[1.0]


In [75]:
import numpy as np

# Assuming 'rfModel' is your trained Random Forest Model

k = 10  # you can change this value depending on how many top features you want

"""
# Get the most important features 'k'
for i in range(k):
    print(f"{X_data.iloc[indices[i]]} ({importances[indices[i]]})")
"""

Feature ranking:


'\n# Get the most important features \'k\'\nfor i in range(k):\n    print(f"{X_data.iloc[indices[i]]} ({importances[indices[i]]})")\n'